## Automating Reasoning in Hoare Logic

### Motivation
- Manually writing proofs for Hoare triples is **tedious**.
- We want to **automate** reasoning by generating **verification conditions (VCs)**.
- We will **assume** loop invariants are provided (by a human or a static analysis tool).

---

## Verification Conditions (VCs)

- A **verification condition** is a **first-order logic formula**:
  - If the VC is valid, the Hoare triple is valid.
- The process:
  1. Analyze the source code.
  2. Generate a VC.
  3. Feed it to a theorem prover.
  4. If valid, the program satisfies its specification.

---

## VC Generation Approaches

### 1. **Forward Reasoning**
- Start with the precondition \( P \), execute program **forward**, compute a **strongest postcondition (SP)**.
- Then check:  
  \( \text{SP} \Rightarrow Q \)

- **Disadvantages**:
  - Can accumulate **irrelevant constraints**.
  - May produce **quantified formulas**.

### 2. **Backward Reasoning (Weakest Preconditions)**
- Start with postcondition \( Q \), work **backward** through the program to compute **weakest precondition (WP)**.
- Then check:  
  \( P \Rightarrow \text{WP} \)

- **Advantages**:
  - More **efficient**.
  - Often results in **quantifier-free** formulas.

---

## What is a Weakest Precondition?

Given a program statement \( S \) and postcondition \( Q \),  
the weakest precondition \( \text{wp}(S, Q) \) is the **least restrictive** condition  
that ensures executing \( S \) leads to \( Q \) holding afterward.

To prove the Hoare triple \( \{P\}\ S\ \{Q\} \),  
compute \( \text{wp}(S, Q) \), then check if \( P \Rightarrow \text{wp}(S, Q) \).

---

## Computing Weakest Preconditions (Non-Loops)

### Assignment

For: `x := e`,  

\[
\text{wp}(x := e, Q) = Q[x \leftarrow e]
\]

Substitute all instances of \( x \) in \( Q \) with \( e \).

---

### Sequencing

For: `S1; S2`,  

\[
\text{wp}(S1; S2, Q) = \text{wp}(S1, \text{wp}(S2, Q))
\]

Go backward: first compute wp for \( S2 \), then plug into \( S1 \).

---

### Conditionals

For: `if C then S1 else S2`,  

\[
\text{wp}(\text{if } C \text{ then } S1 \text{ else } S2, Q) = (C \Rightarrow \text{wp}(S1, Q)) \land (\neg C \Rightarrow \text{wp}(S2, Q))
\]

---

### Example

Code snippet:

```

x := y + 1
if x > 0 then z := 1 else z := -1

```

Goal: Prove \( z > 0 \)

1. Then-branch wp: \( 1 > 0 = \text{true} \)  
2. Else-branch wp: \( -1 > 0 = \text{false} \)  

So conditional wp:  
\[
(x > 0 \Rightarrow \text{true}) \land (x \leq 0 \Rightarrow \text{false}) = x > 0
\]

Now go one step further back:  
\[
x := y + 1 \Rightarrow \text{wp} = y + 1 > 0
\]

---

## Loops: The Hard Part

You **cannot compute** wp for loops exactly — they may run indefinitely.

### Solution:
Use **loop invariants** and approximate weakest preconditions.

Notation:  
\[
\text{awp}(\text{while } C\ \text{do } S\ [I], Q) = I
\]

Where \( I \) is the **loop invariant**.

---

## Loop Verification Conditions

We generate **3 VCs**:

### 1. Initial condition

Prove:  
\[
P \Rightarrow I
\]

We must establish that the invariant holds before the loop.

---

### 2. Inductiveness

Prove:  
\[
\{I \land C\}\ S\ \{I\}
\]

Executing the body preserves the invariant.

---

### 3. Postcondition after the loop

Prove:  
\[
I \land \neg C \Rightarrow Q
\]

When the loop terminates, the invariant + termination imply the postcondition.

---

## Summary

To verify a Hoare triple \( \{P\}\ S\ \{Q\} \):

- Generate \( \text{wp}(S, Q) \) (or \( \text{awp} \) for loops)
- Check: \( P \Rightarrow \text{wp}(S, Q) \)
- For **every loop**, check:
  1. \( P \Rightarrow I \)
  2. \( \{I \land C\}\ S\ \{I\} \)
  3. \( I \land \neg C \Rightarrow Q \)

If all hold, the Hoare triple is valid.

---
